# Introduction

## Importance of Differentiation 
**Differentiation** is one of the most commonly used mathematical operations. Fundamentally, it computes the rate of change of a variable with respect to another. This rate is referred to as the derivative of the first variable with respect to the second. Differentiation is also applied to single or multi-valued functions to compute their derivatives with respect to one or more dependent variables. Values of these derivatives serve as a critical input to several numerical algorithms (e.g. Newton's method) and considering the fast growing complexity of the computational problems being solved these days it is imperative to compute these derivatives accurately as well as quickly. 

## Automatic Differentiation
**Automatic Differentiation (abbreviation: AD and also referred to as Algorithmic or Computational Differentiation)** is a method to evaluate derivatives of real-valued functions. It is a variant of the classically conceptualized computer-based differentiation methods such as symbolic differentation and finite difference methods. It addresses shortcomings encountered in approaches such as symbolic and finite-difference differentiation. For example, symbolic differentation possesses the capability to compute derivatives to machine precision, however the required computational times can be quite large. On the other hand, finite difference differentiation is quicker but errors in the computed derivatives are several order of magnitudes higher than machine precision. AD emerges as a solution devoid of shortcomings observed in both symbolic and finite difference methods and hence is gaining popularity in computational scientific applications.

## Applications of Automatic Differentiation
Automatic differentiation can be applied towards solving multiple important computational problems. Examples include root finding methods such as the Newton's method, optimization schemes such as the Gradient Descent method, and machine learning algorithms such as the backpropagation algorithm. All these methods have a critical dependence on quick computation of accurate derivates of the function being considered. Automatic differentiation serves these requirements and hence is an appropriate choice for calculating the derivatives for these algorithms.

## The Gradient Descent Method 
The **Gradient Descent Method (GDM)** is an iterative optimization algorithm. It can be used to find the local minima of a differentiable function by iteratively miniminizing a cost function and ultimately stopping at the minimum value of the cost function which indicates a local minima.

In 1-D the following steps are performed in GDM.
**Step 1**: Given a function in x, start at a random point x0

**Step 2**: Compute the derivative at x0

**Step 3**: Check if the algorithm is going in the right direction based on the sign of the derivative or have we reached the minima

**Step 4**: Get a new value of x and repeat Steps 1-3 until converge to a local minima

This approach can be extended to higher dimensions.

## Scope of the Current Software Package
The software package presented here provides a computational tool to compute derivatives of multivalued-functions employing the AD technique. The package will compute derivatives using the forward AD method. In addition, it would also include the capability of performing optimization using the GDM. 


## References
1. Hoffmann, P. H. (2016). A hitchhiker’s guide to automatic differentiation. Numerical Algorithms, 72(3), 775-811.
2. van Merrienboer, B., Moldovan, D., & Wiltschko, A. (2018). Tangent: Automatic differentiation using source-code transformation for dynamically typed array programming. In Advances in Neural Information Processing Systems (pp. 6256-6265).
3. https://harvard-iacs.github.io/2020-CS107/lectures/.
4. Griewank, A. and Walther, A., 2008. Evaluating derivatives: principles and techniques of algorithmic differentiation (Vol. 105). Siam.
5. Nocedal, J. and Wright, S., 2001. Numerical Optimization, Second Edition. Springer.

# Background

## The Chain Rule

Automatic differentitation is built upon the chain rule, which provides a formula to compute the derivative of composite functions. Recall that if we have a composite function

$$ f(g(h(x))), $$

the chain rule tells us that

$$ \frac{\partial f}{\partial x} = \frac{\partial f}{\partial g} \frac{\partial g}{\partial h}\frac{\partial h}{\partial t}. $$

In this package, we implement the forward mode of AD, whereby we evaluate the derivative of a composite function from the inside out. Put differently, we differentiate with respect to one independent variable and recursively find the derivative of each sub-expression starting with the innermost function. In general, the forward mode would evaluate the derivative above as follows:

$$ \frac{\partial f}{\partial x} = \frac{\partial f}{\partial v_{n-1}} \frac{\partial v_{n-1}}{\partial x} = \frac{\partial f}{\partial v_{n-1}}  \left( \frac{\partial v_{n-1}}{\partial v_{n-2}}\frac{\partial v_{n-2}}{\partial x} \right) = \frac{\partial f}{\partial v_{n-1}} \frac{\partial v_{n-1}}{\partial x} = \frac{\partial f}{\partial v_{n-1}}  \left( \frac{\partial v_{n-1}}{\partial v_{n-2}} \left( \frac{\partial v_{n-2}}{\partial v_{n-3}} \frac{\partial v_{n-3}}{\partial x} \right) \right) = ..., $$

where each $ v_i $ is an inner function. If the function we want to differentiate has multiple arguments, as in

$$ f(g(t), h(t)), $$

then the chain rule extends naturally, giving

$$ \frac{\partial f}{\partial x} = \frac{\partial f}{\partial g}\frac{\partial g}{\partial x} + \frac{\partial f}{\partial h}\frac{\partial h}{\partial x}. $$

And if we have a vector-valued function with $ x \in \mathbb{R}^m $

$$ f = f(g(x), h(x)), $$

the derivative becomes

$$ \nabla_x f = \frac{\partial f}{\partial g} \nabla g + \frac{\partial f}{\partial h} \nabla h. $$

Putting this all together, we can use the chain rule to express the derivative of a general vector-valued function $ f=f(y(x)) $ where $ y \in \mathbb{R}^n $ and $ x \in \mathbb{R}^m $ as

$$ \nabla_x f = \sum_{i=1}^n \frac{\partial f}{\partial y_i} \nabla y_i(x). $$


## The Computational Graph

Consider the function $ f(x, y) = e^{x^2 + y^2}. $ For illustrative purposes, we can summarize the recursive operations of forward mode AD in a computational graph. 

Let's say we want to find $ \partial f / \partial x. $ Because we are differentiating with respect to $ x, $ we start by calculating seed values as:

$$ \frac{\partial x}{\partial x} = 1, $$ and
$$ \frac{\partial y}{\partial x} = 0. $$

Then the derivative is evaluated as follows:

| Trace   | Elementary Operation   | Derivative Operation                     |
|---------|------------------------|------------------------------------------|
| $v_1$   | $x$                    | $\dot{v_1}=1$ (seed)                     |
| $v_2$   | $v_1^2$                | $\dot{v_2}=2v_1 \dot{v_1}$               |
| $v_3$   | $y$                    | $\dot{v_3}=0$ (seed)                     |
| $v_4$   | $v_3^2$                | $\dot{v_4}=2v_3 \dot{v_3}$               |
| $v_5$   | $v_2 + v_4$            | $\dot{v_5}=\dot{v_2} + \dot{v_4}$        |
| $f$     | $e^{v_5}$              | $\partial f/\partial x=\dot{v_5}e^{v_5}$ |

In essence, forward mode AD does computationally what we did by hand in the table above. In general, it computes the product $\nabla f \cdot p,$ where $ p $ is a seed vector computed as we did in the example above. If $ f $ is a vector-valued function, then it computes $ Jp, $ where $ J $ is the Jacobian.

# Installation Process
We will discuss on this as we implement the design since it would be clear what dependencies (other libraries) we are using and it would be easier to list the installation process and the steps involved. We will update this section as we have further updates. 

# Software Organization

## Directory Structure
Provided below is a proposed directory structure for the AD library.

### Directory 1
++--source_code 
|   +-- AutoDiffpy.py
|   +-- GradDescpy.py
|   +-- Testscript.py
### Directory 2
++--README  
|   +-- README.md
### Directory 3
++--Documentation  
|   +-- doc_files
### Directory 4
++--Set_up  
|   +-- set_up_instructions_file


## Python Modules to be used
The Python modules that can be used are:
- numpy for accessing user-input functions.
- math for performing elementary mathematical operations.

## Test Suite to be used
The package will be tested using the TravisCI testing tool. The Github repository of this package has already been linked to TravisCI as part of milestone 1b.

## Package Distribution
The initial plan is to distribute the package on a Github repository with access available to clone and run it. The accompanying directories in the package will contain detailed instructions on how to run the package.

## Framework for Software
Currently there is no plan of using a framework. 

## Sample Structure of the Package
Figure 1 shows a use case diagram for the software package. The actors in the diagram include the user and the two classes (AutoDiffpy and GradDespy) to perform the autodifferentiation and optimization respectively. The user can use the package to just differentiate the function or to find the optimum value of a function using GDM.  

![Use_Case_Diagram](UseCaseDiagram0.png)
**Figure 1: Use case diagram for the package**

Figure 2 shows a class diagram for the package. The package consists of only two classes **(AutoDiffpy and GradDespy)** which contain methods to perform the autodifferentiation and optimization respectively. There is also a testscript which acts as an API. It will read the user input, create the appropriate object (AD or GDM), and use the properties of the object (methods and attributes) to perform the required tasks (i.e AD or optimization using AD).The current implementation includes python lists as the only data structures used for storage. However, this is preliminary and can be changed once development begins.

![Class_Diagram](ClassDiagram0.png)
**Figure 2: Class diagram for the package**

Provided below is a very basic structure of the source code of this package. This structure is still tentative and can be revised based on further feedback and discussion.

## Class AutoDiffpy
This is the class to perform AD forward mode. It contains a method to parse the user input function and get which basic functions exist (e.g. sine, cosine) and how many variables there are (x, (x,y), (x,y,z)). The AD object would have properties such as Jacobian etc. associated with it. The method get_Jacobian could be used to compute the Jacobian matrix while the method get_derivative could be used to compute the required derivative.

## Class GradDescpy
This is the class to perform the GDM. It also contains a method to parse the user input function. It also has methods to compute the objective function (compute_ObjFunc) and get the function values at different iterations (get_funcValues). These would again be properties associated with the GD object and can be provided to the user once the computation is done.

## Class Testscript
This is the API class which prompts the user for input and then calls the appropriate classes (AD or GradDesc) to perform the required tasks.

In [ ]:
class AutoDiffPy:
    
    # Initialize the constructor for an AD object
    def __init__():
        pass
    
    # Function to parse the user input function
    def parse_input(user_input):
        pass
    
    # Function to implement forward automatic differentiation
    def get_Jacobian():
        pass
    
    def get_Derivative():
        pass
class GradDescPy:

    def __init__(parameters, cost_function, optimization_function):
        params = parameters
        cost_function = cost_function
        optimization_function = optimization_function

    # Function to implement backward automatic differentiation
    def optimizer(self.params, self.cost_function, self.optimization_function):
        pass
        
    def set_params(parameters):
        params = parameters
    
    def get_params():
        return self.params

    

# How to Use ad_library

1. The user will install the ad_library library. 
2. Once this is complete, they can import the module ad_library which will give them access to the AutoDiffpy and GradDescpy classes. 
<br>
3a. The user will be able to construct an AutoDiffpy object in order to perform forward automatic differentation operations on a given function.
<br>
3b. The user will be able to construct an GradDescpy class in order to perform optimization using a gradient descent algorithm.
<br>
Below we list some pseudocode to demonstrate general use cases.


In [ ]:
import ad_library

# Part 1: Forward Automatic Differentation

# User defines function on which to run forward automatic differentation
my_function = sample_function

# Construct ad_forward class
ad = AutoDiffpy()

# Run forward automatic differentation on a sample function
user_forward = ad.get_Jacobian(input_function, seed_vector = [1, 1])
user_forward = ad.get_Derivative(input_function)

# Part 2: Optimization via Gradient Descent
params = sample_params
optimization_function = sample_of
vector = sample_vector

# Construct ad_optimizer class
ad_opt = GradDescpy(params, optimization_function)

# Run optimization
optimized = ad_opt.optimizer(vector)




# Implementation

## Core Data Structures:
Arrays: These will be necessary throughout optimization and forward automatic differentation. We will be using numpy arrays to represent vectors.
Lists: These are necessary and will also be implemented using both the Python built in lists and numpy.


## Classes:
The two primary classes within this library will be the ad_forward class which will contain the methods necessary to perform forward automatic differentiation, while the ad_optimizer class will be used to perform optimization via the gradient descent algorithm. Their methods and attributes are described below.

### Method and name attributes:
The forward_ad object will contain the methods necessary to perform forward automatic differentiation and optimization. These will contained in their own methods with a few additional helper methods. These are listed below with descriptions.

#### AutoDiffPy
- __init__()
    - Parameters: 0 Parameters
    - Returns: A constructed ad_library object
- parse_function(user_input)
    - Parameters: A raw user input representing a function
    - Returns: The parsed function to input into forward_ad 
- get_Jacobian(input_function, seed_vector = default)
    - Parameters: An input function and an optional seed vector parameter.
    - Returns: The result of forward automatic differentation on the given function.
- get_Derivative(input_function)
    - Parameters: An input function
    - Returns: Derivative of input function

#### GradDescPy
Attributes:
- params
     - params contains vector of coefficients
- optimization_function
    - The optimization function is the optimization function for gradient descent
- cost_function_value
    - This contains the value of the cost function

Methods:
- __init__(parameters, optimization_function = default_of)
    - Parameters: Parameters which include an array representing the vector, and optional parameters which define the optimization function
    - Returns: A constructed ad_optimizer object to perform optimization via gradient descent.
- parse_function(user_input)
    - Parameters: Raw user input representing a function.
    - Returns: The parsed function to input into the optimize function 
- optimizer(vector)
    - Performs gradient descent given a cost function and a set of vectors
- get_params()
- set_params()


## External Dependencies:
- Numpy
    - Numpy will be used in order to be build highly dynamic arrays. This will be necessary often throughout our implementation in order to...
- Math
    - The Math package will be used to deal with elementary functions and operations. This is necessary throughout differentiation in all modes.

## Dealing with elementary functions
These will all be implemented using the python math module which includes:
- math.sin()
- math.sqrt()
- math.log()
- math.exp()
- An expansive list of elementary functions, numbers, and operations that can be accessed through the math module.

Dealing with edge use cases:
- Scalars and Vectors
    - We will deal with vector inputs using numpy arrays which are able to handle the operations necessary to perform the necessary functions.
    - Secondarily, we can simply account for these vector valued functions in the parse input function. We can use this function, to identify the number of variables, whether it is a vector, and parse it appropriately.
    - We will deal with vector inputs using numpy arrays which are able to handle the operations necessary to perform the necessary functions.
    - While not completely an edge case, we will deal with basic operations by overloading the add and mult operators.

# Feedback on Background
Going forward, I would also like to see more discussion on what forward mode actually computes (Jacobian-vector product), the "seed" vector, and the efficiency of forward mode.

# Feedback Response
Sure. We will add more discussion.

# Feedback on How to use
It would be nice to include the installation process. Please put this section before Software Organization

# Feedback Response
We have added a section on installation process. We will fill it out as we continue our implementation and development.

# Feedback on implementation
The following were the critiques and questions we were asked to address from our milestone 1 submission.

- Your implementation for vector-valued functions is unclear.
- Will you implement operator overloading methods?

# Feedback Response
In order to address these critiques, we did the following:

 - To deal with vector valued functions, we decided to add an additional parameter to the construction of our two classes, AutoDiffPy and GradDescPy at init(). We added a parameter which takes in the number of independent variables which allows us to identify if we will need to account for vector valued functions.

- Alternatively, we are considering a version of our implementation which asks less of the user, and is able to parse this function to identify whether or not it is a vector valued function in this parse_function() method we have in both classes. This may require usage of the symbolic python library in order to parse inputs in a generalized format and perform the necessary vector or non-vector operations on it.

- We agreed that it was necessary to implement operator overloading methods. We added in the add and mul operator overloading methods such that our classes could handle these operations. We additionally, would account for both right and left operations by adding overloading the radd and rmul operators as well.